This notebook plots the LSST redshift distribution, the Y1 source tomographic bins, and the Lyman dropout distributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
from scipy.ndimage import gaussian_filter

In [ ]:
def nz(z, i):
    z0 = 0.0417 * i - 0.744
    n = 1 / (2 * z0) * (z / z0)**2 * np.exp(-z/z0)
    n /= np.sum(n)
    return n

In [ ]:
def plot_gaussian(ax, x, mu, sig, name, color):
    # calculate the Gaussian
    gaussian = np.exp(-(x - mu)**2 / (2 * sig**2)) / np.sqrt(2 * np.pi * sig**2)

    # plot the Gaussian
    ax.plot(x, gaussian, c=color)

    # label the Gaussian
    offset = 0.25 if name == "g" else 0.15
    ax.text(mu, gaussian.max() + offset, name, ha="center", va="bottom", color=color)


def plot_distributions(
        highz=False,
        log=False,
        wl_sources=False,
        lya_dropouts=False,
        barrier=False,
    ):
    # 20 year 5-sigma limiting i magnitude
    i20 = 26.87

    # total number of galaxies in the sample
    Ntot = 11.5e9

    # create the figure
    figsize = (7, 4.2)
    dpi = 120
    if wl_sources or lya_dropouts:
        fig, axes = plt.subplot_mosaic(
            """
            A
            A
            B
            B
            B
            B
            """,
            figsize=figsize, 
            dpi=dpi,
        )
    else:
        fig, axes = plt.subplot_mosaic(
            """
            B
            B
            B
            B
            """,
            figsize=figsize, 
            dpi=dpi,
        )

    # some label settings
    if not log:
        ylabel = "Number of galaxies ($x10^9$)"
        scale = 1e9
    else:
        ylabel = "Number of galaxies"
        scale = 1

    # plot the redshift histogram
    ax = axes["B"]
    zmin = 0
    zmax = 8 if highz else 3.5
    dz = 0.25
    zbins = np.arange(zmin, zmax + dz, dz)
    z = (zbins[:-1] + zbins[1:]) / 2
    Nz = Ntot * nz(z, i20)
    ax.hist(z, weights=Nz/scale, bins=zbins, log=log, color="cornflowerblue")

    # axis settings
    ax.set(
        xlim=(zmin, zmax), 
        ylim=(1e3/scale, None), 
        xlabel="Redshift", 
        ylabel=ylabel,
    )

    # plot the distributions on top
    if wl_sources or lya_dropouts:
        ax = axes["A"]

        # axis settings
        ax.set(
            xticks=[], yticks=[], ylabel="$n(z)$", xlim=(zmin, zmax), ylim=(0.05, 5.4)
        )

    # plot the Y1 source distributions
    if wl_sources:
        # calculate n(z) on a fine grid
        zfine = np.linspace(0, 4, 1000)
        nzfine = nz(zfine, 24.1)

        # separate redshift range into bins with equal numbers of galaxies
        nbins = 5
        bin_idx = np.abs(
            nzfine.cumsum()[None, :] - np.linspace(0, 1, nbins + 1)[:, None]
        ).argmin(axis=1)

        # loop over the source
        for i in range(nbins):
            # get the true n(z) for the bin
            nzbin = nzfine.copy()
            nzbin[:bin_idx[i]] = nzbin[bin_idx[i+1]:] = 0

            # calculate the photo-z error for the bin
            zmean = zfine[nzbin > 0].mean()
            sigma = 0.03 * (1 + zmean) / (zfine[1] - zfine[0])

            # convolve the true distribution with the Gaussian error
            nzbin = gaussian_filter(nzbin, sigma)

            # normalize n(z) for the bin
            nzbin /= np.trapz(nzbin, zfine)

            # plot the distribution
            ax.plot(zfine, nzbin, c="k", lw=1)

    # plot the expected distribution of Lyman Dropouts
    if lya_dropouts:
        # create a redshift range that covers all the dropouts
        zfine = np.linspace(0, 8, 1000)

        # plot the u band dropouts
        u_mu = 2.96
        u_FWHM = 0.7
        plot_gaussian(ax, zfine, u_mu, u_FWHM / 2.355, "u", "C4")

        # plot the g band dropouts
        g_mu = 3.8
        g_FWHM = 4.3 - 3.3
        plot_gaussian(ax, zfine, g_mu, g_FWHM / 2.355, "g", "C0")

        # plot the r band dropouts
        r_mu = 4.9
        r_FWHM = 5.15 - 4.5
        plot_gaussian(ax, zfine, r_mu, r_FWHM / 2.355, "r", "C2")

        # plot the i band dropouts
        i_mu = 5.9
        i_FWHM = 6.15 - 5.7
        plot_gaussian(ax, zfine, i_mu, i_FWHM / 2.355, "i", "C3")

        # plot the z band dropouts
        z_mu = 6.9
        z_FWHM = 7.5 - 6.45
        plot_gaussian(ax, zfine, z_mu, z_FWHM / 2.355, "z", "tomato")

    if barrier:
        # put the barrier and text on the bottom panel
        ax = axes["B"]
        ax.axvline(2, c="k", lw=1)
        if log:
            ax.text(1.85, 1e5, "3x2pt\ncosmology", ha="right", color="w")
            ax.annotate(
                "",
                xy=(0.6, 0.35e5),
                xytext=(1.85, 0.35e5),
                arrowprops=dict(arrowstyle="->", color="w"),
            )

            ax.text(2.15, 1e5, "high-redshift\ngalaxies", ha="left", color="w")
            ax.annotate(
                "",
                xy=(3.4, 0.35e5),
                xytext=(2.15, 0.35e5),
                arrowprops=dict(arrowstyle="->", color="w"),
            )

        # put the barrier and text on the top panel
        if wl_sources or lya_dropouts:
            ax = axes["A"]
            ax.axvline(2, c="k", lw=1)
        if wl_sources:
            ax.text(1.9, 5, "3x2pt\nsources", ha="right", va="top", fontsize=8)
        if lya_dropouts:
            ax.text(2.1, 5, "Lyman dropouts", ha="left", va="top", fontsize=8)

        # print the number of galaxies above the panels
        trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.text(
            1,
            1.05,
            "10 billion",
            fontsize=9,
            transform=trans,
            ha="center",
            va="bottom",
        )
        ax.text(
            (2+zmax)/2, 1.05,
            "1 billion galaxies",
            fontsize=9,
            transform=trans,
            ha="center",
            va="bottom",
        )

        # if we didn't plot the Lyman Dropouts, print a question mark
        if not lya_dropouts:
            ax.text(
                (2+zmax)/2,
                0.5,
                "?",
                fontsize=16,
                transform=trans,
                ha="center",
                va="center",
            )
    elif highz:
        ax.set_title("11 billion galaxies", fontsize=9)
    else:
        ax.set_title("10 billion galaxies", fontsize=9)

    plt.tight_layout()
    plt.subplots_adjust(hspace=0)

    return fig, ax

In [ ]:
fig, ax = plot_distributions(highz=False, log=False, wl_sources=True, lya_dropouts=False, barrier=False)
fig.savefig("figures/nz+wlSources.png", dpi=500)

In [ ]:
fig, ax = plot_distributions(highz=True, log=True, wl_sources=True, lya_dropouts=False, barrier=True)
fig.savefig("figures/nz+wlSources_highz.png", dpi=500)

In [ ]:
fig, ax = plot_distributions(highz=True, log=True, wl_sources=True, lya_dropouts=True, barrier=True)
fig.savefig("figures/nz+wlSources+dropouts_highz.png", dpi=500)